In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在GCP上进行端到端的ML：MLOps阶段2：使用Vertex AI实验开始自动记录TensorFlow模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_vertex_experiments_autologging_tf.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_vertex_experiments_autologging_tf.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_with_vertex_experiments_autologging_tf.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用DIY代码使用`Vertex AI Experiments`来实现对实验参数和指标的自动记录。

### 目标

在本教程中，您将学习如何创建一个用于训练 TensorFlow 模型的实验，并使用附带的自助（DIY）代码自动记录参数和指标。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI Experiments`

执行的步骤包括：

- 构建自助记录代码。
- 构建具有调用自助记录功能的 TensorFlow Sequential 模型的训练包。
- 训练模型。
- 查看实验结果。
- 构建具有调用自助记录功能的 TensorFlow Functional 模型的训练包。
- 比较实验结果。
- 删除实验。

数据集

本教程使用的数据集是[Boston房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。该教程使用的数据集版本已经整合到TensorFlow中。训练好的模型可以预测房屋价格的中位数，单位为1千美元。

费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成费用估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex Workbench AI笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简要的说明：

1. [安装和初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装以下软件包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade --quiet {USER_FLAG} google-cloud-aiplatform \
                                             tensorflow==2.5 \
                                             numpy

### 重新启动内核

在安装了额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

此教程不需要GPU运行时。

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用，可用于支付计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下API：Vertex AI APIs、Compute Engine APIs和Cloud Storage](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]。

5. 在下面的单元格中输入您的项目ID。然后运行此单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并且会以`$`为前缀的Python变量进行内插操作。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改 `REGION` 变量，该变量用于本笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有的Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您参加的是实时教程会话，您可能正在使用共享的测试账户或项目。为避免在创建的资源上发生名称冲突，您需要为每个实例会话创建一个UUID，并将其附加到本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过身份验证。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行oAuth身份验证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入一个名称，然后点击**创建**。

在**授予此服务账号对项目的访问权限**部分，点击角色下拉列表。在筛选框中键入"Vertex"，然后选择**Vertex管理员**。在筛选框中键入"存储对象管理员"，然后选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中将您的服务账号密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import numpy as np
import tensorflow as tf

初始化用于您的项目和相应存储桶的 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## TensorFlow Keras模型自动记录的DIY代码

下面的代码使用Keras API 实现了对TensorFlow模型的自动记录。

- `autologging()`: 初始化实验并使用堆注入来替换堆上的TF.keras `Sequential`和`Model`符号，分别用重定向包装类`VertexTFSequential`和`VertexTFModel`。

- `VertexTFSequential`: tf.keras.Sequential类的子类。
    - `compile()`: 覆盖了超类的方法。自动记录指定的超参数并调用底层的`compile()`方法。
    - `fit()`: 覆盖了超类的方法。自动记录指定的超参数，调用底层的`fit()`方法，并记录结果指标。
    - `evaluate()`: 覆盖了超类的方法。调用底层的`evaluate()`方法，并记录结果指标。
- `VertexTFModel`: tf.keras.Model类的子类。
- `VertexTFHelper`: 用于Sequential和Functional模型的通用记录方法的类。

In [ ]:
def autolog(
    project: str = None,
    location: str = None,
    staging_bucket: str = None,
    experiment: str = None,
    run: str = None,
    framework: str = "tf",
):
    """
    Enable automatic logging of parameters and metrics in Vertex AI Experiments,
    for corresponding framework.

        project: The project ID
        location : The region
        staging_bucket: temporary bucket
        experiment: The name of the experiment
        run: The name of the run within the experiment
        framework: The ML framework for which a model is being trained.
    """
    # autologging
    if framework == "tf":
        try:
            globals()["Sequential"] = VertexTFSequential
            if "tf" in globals():
                tf.keras.Sequential = VertexTFSequential
            if "tensorflow" in globals():
                tensorflow.keras.Sequential = VertexTFSequential
        except:
            pass

        try:
            globals()["Model"] = VertexTFModel
            if "tf" in globals():
                tf.keras.Model = VertexTFModel
            if "tensorflow" in globals():
                tensorflow.keras.Model = VertexTFModel
        except:
            pass

    if project:
        aiplatform.init(
            project=project, location=location, staging_bucket=staging_bucket
        )

    if experiment:
        aiplatform.init(experiment=experiment)
    if run:
        aiplatform.start_run(run)


class VertexTFSequential(tf.keras.Sequential):
    """
    Sublass of the tensorflow.keras.Sequential model type. Overrides with pass thru of
    underlying super class methods to automatically log parameters/metrics for Vertex AI experiments.

        compile():
        fit():
        evaluate():
    """

    def __init__(self, layers):
        return super().__init__(layers)

    def compile(
        self,
        optimizer="rmsprop",
        loss=None,
        metrics=None,
        loss_weights=None,
        weighted_metrics=None,
        run_eagerly=None,
        steps_per_execution=None,
    ):
        try:
            learning_rate = optimizer.learning_rate.numpy()
            aiplatform.log_params({"train.learning_rate": float(learning_rate)})
        except:
            pass
        return super().compile(
            loss=loss,
            optimizer=optimizer,
            metrics=metrics,
            loss_weights=loss_weights,
            weighted_metrics=weighted_metrics,
            run_eagerly=run_eagerly,
            steps_per_execution=steps_per_execution,
        )

    def fit(
        self,
        x=None,
        y=None,
        batch_size=None,
        epochs=1,
        verbose="auto",
        callbacks=None,
        validation_split=0.0,
        validation_data=None,
        shuffle=True,
        class_weight=None,
        sample_weight=None,
        initial_epoch=0,
        steps_per_epoch=None,
        validation_steps=None,
        validation_batch_size=None,
        validation_freq=1,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
    ):
        aiplatform.log_params({"train.epochs": int(epochs)})
        if batch_size:
            aiplatform.log_params({"train.batch_size": int(batch_size)})
        if steps_per_epoch:
            aiplatform.log_params({"train.steps": int(steps_per_epoch)})

        history = super().fit(
            x=x,
            y=y,
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose,
            callbacks=callbacks,
            validation_split=validation_split,
            validation_data=validation_data,
            shuffle=shuffle,
            class_weight=class_weight,
            sample_weight=sample_weight,
            initial_epoch=initial_epoch,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            validation_batch_size=validation_batch_size,
            validation_freq=validation_freq,
            max_queue_size=max_queue_size,
            workers=workers,
            use_multiprocessing=use_multiprocessing,
        )

        TFHelper().model_size(self)

        for key, val in history.history.items():
            aiplatform.log_metrics({f"train.{key}": val[-1]})
        return history

    def evaluate(
        self,
        x=None,
        y=None,
        batch_size=None,
        verbose=1,
        sample_weight=None,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
        return_dict=False,
    ):

        metrics = super().evaluate(
            x=x,
            y=y,
            batch_size=batch_size,
            verbose=verbose,
            sample_weight=sample_weight,
            steps=steps,
            callbacks=callbacks,
            max_queue_size=max_queue_size,
            workers=workers,
            use_multiprocessing=use_multiprocessing,
            return_dict=return_dict,
        )

        aiplatform.log_metrics({"eval.loss": metrics[0]})
        for _ in range(1, len(metrics)):
            aiplatform.log_metrics({"eval.metric": metrics[_]})
        return metrics


class VertexTFModel(tf.keras.Model):
    """
    Sublass of the tensorflow.keras.Model model type. Overrides with pass thru of
    underlying super class methods to automatically log parameters/metrics for Vertex AI experiments.

        compile():
        fit():
        evaluate():
    """

    def __init__(self, inputs, outputs):
        return super().__init__(inputs, outputs)

    def compile(
        self,
        optimizer="rmsprop",
        loss=None,
        metrics=None,
        loss_weights=None,
        weighted_metrics=None,
        run_eagerly=None,
        steps_per_execution=None,
    ):
        try:
            learning_rate = optimizer.learning_rate.numpy()
            aiplatform.log_params({"train.learning_rate": float(learning_rate)})
        except:
            pass
        return super().compile(
            loss=loss,
            optimizer=optimizer,
            metrics=metrics,
            loss_weights=loss_weights,
            weighted_metrics=weighted_metrics,
            run_eagerly=run_eagerly,
            steps_per_execution=steps_per_execution,
        )

    def fit(
        self,
        x=None,
        y=None,
        batch_size=None,
        epochs=1,
        verbose="auto",
        callbacks=None,
        validation_split=0.0,
        validation_data=None,
        shuffle=True,
        class_weight=None,
        sample_weight=None,
        initial_epoch=0,
        steps_per_epoch=None,
        validation_steps=None,
        validation_batch_size=None,
        validation_freq=1,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
    ):
        aiplatform.log_params({"train.epochs": int(epochs)})
        if batch_size:
            aiplatform.log_params({"train.batch_size": int(batch_size)})
        if steps_per_epoch:
            aiplatform.log_params({"train.steps": int(steps_per_epoch)})

        history = super().fit(
            x=x,
            y=y,
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose,
            callbacks=callbacks,
            validation_split=validation_split,
            validation_data=validation_data,
            shuffle=shuffle,
            class_weight=class_weight,
            sample_weight=sample_weight,
            initial_epoch=initial_epoch,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            validation_batch_size=validation_batch_size,
            validation_freq=validation_freq,
            max_queue_size=max_queue_size,
            workers=workers,
            use_multiprocessing=use_multiprocessing,
        )

        TFHelper().model_size(self)

        for key, val in history.history.items():
            aiplatform.log_metrics({f"train.{key}": val[-1]})
        return history

    def evaluate(
        self,
        x=None,
        y=None,
        batch_size=None,
        verbose=1,
        sample_weight=None,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
        return_dict=False,
    ):

        metrics = super().evaluate(
            x=x,
            y=y,
            batch_size=batch_size,
            verbose=verbose,
            sample_weight=sample_weight,
            steps=steps,
            callbacks=callbacks,
            max_queue_size=max_queue_size,
            workers=workers,
            use_multiprocessing=use_multiprocessing,
            return_dict=return_dict,
        )

        aiplatform.log_metrics({"eval.loss": metrics[0]})
        for _ in range(1, len(metrics)):
            aiplatform.log_metrics({"eval.metric": metrics[_]})
        return metrics


class TFHelper(object):
    def model_size(self, model):
        """
        Get the memory footprint as measured by the number of weights
        """

        def get_size(weights) -> int:
            n = 0
            for weight in weights:
                try:
                    n += len(weight)
                    n += get_size(weight)
                except:
                    pass

            return n

        n = get_size(model.get_weights())
        aiplatform.log_metrics({"n_weights": n})
        return n

使用Vertex AI实验训练一个TensorFlow Sequential模型

在下面的代码中，您会构建、训练和评估一个TensorFlow Sequential的表格模型。Python脚本包括以下调用以集成`Vertex AI实验`：

- 命令行参数：参数`experiment`和`run`用来传递实验和运行名称。
- `autologging()`: 初始化实验并进行堆注入。
- `aiplatform.start_execution()`: 初始化一个上下文用于链接工件。
- `aiplatform.end_run()`: 结束实验。

*注意:* 初始化器`Sequential`会被堆注入重定向到`VertexTFSequential`。当对compile()、fit()和evaluate()方法进行后续调用时，它们将作为相应的`VertexTFSequential`方法执行。

In [ ]:
EXPERIMENT_NAME = f"myexperiment{UUID}"
RUN_NAME = "run-1"


def make_dataset():

    # Scaling Boston Housing data features
    def scale(feature):
        max = np.max(feature)
        feature = (feature / max).astype(np.float)
        return feature, max

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
        path="boston_housing.npz", test_split=0.2, seed=113
    )
    params = []

    for _ in range(13):
        x_train[_], max = scale(x_train[_])
        x_test[_], _ = scale(x_test[_])
        params.append(max)

    return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model(lr):
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(128, activation="relu", input_shape=(13,)),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1, activation="linear"),
        ]
    )

    model.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    return model


# autologging
autolog(experiment=EXPERIMENT_NAME, run=RUN_NAME)

with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    BATCH_SIZE = 16

    model = build_and_compile_dnn_model(lr=0.01)

    # Train the model
    (x_train, y_train), (x_test, y_test) = make_dataset()
    model.fit(x_train, y_train, epochs=10, batch_size=BATCH_SIZE)

    model.evaluate(x_test, y_test)

aiplatform.end_run()

#### 获取实验结果

接下来，您可以使用实验名称作为参数传递给`get_experiment_df()`方法来获取实验结果作为pandas数据框。

In [ ]:
experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

### 使用Vertex AI Experiments训练一个TensorFlow功能性模型

在以下代码中，您将建立、训练和评估一个TensorFlow功能性表格模型。Python脚本包括以下调用以集成`Vertex AI Experiments`：

- 命令行参数: 参数`experiment`和`run`用于传递实验和运行名称给实验。
- `autologging()`: 初始化实验并进行堆注入。
- `aiplatform.start_execution()`: 初始化一个上下文以链接工件。
- `aiplatform.end_run()`: 结束实验。

*注意:* 初始化器`Model`将通过堆注入重定向到`VertexTFModel`。当对compile()、fit()和evaluate()方法进行后续调用时，它们将作为相应的`VertexTFModel`方法执行。

In [ ]:
RUN_NAME = "run-2"


def make_dataset():

    # Scaling Boston Housing data features
    def scale(feature):
        max = np.max(feature)
        feature = (feature / max).astype(np.float)
        return feature, max

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
        path="boston_housing.npz", test_split=0.2, seed=113
    )
    params = []

    for _ in range(13):
        x_train[_], max = scale(x_train[_])
        x_test[_], _ = scale(x_test[_])
        params.append(max)

    return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model(lr):
    inputs = tf.keras.Input(shape=(13,))
    x = tf.keras.layers.Dense(128, activation="relu")(inputs)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1, activation="linear")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    return model


# autologging
autolog(experiment=EXPERIMENT_NAME, run=RUN_NAME)

with aiplatform.start_execution(
    schema_title="system.ContainerExecution", display_name="example_training"
) as execution:
    BATCH_SIZE = 16

    model = build_and_compile_dnn_model(lr=0.01)

    # Train the model
    (x_train, y_train), (x_test, y_test) = make_dataset()
    model.fit(x_train, y_train, epochs=10, batch_size=BATCH_SIZE)

    model.evaluate(x_test, y_test)

aiplatform.end_run()

#### 获取实验结果

接下来，您可以将实验名称作为参数传递给方法`get_experiment_df()`，以将实验结果获取为pandas dataframe。

In [ ]:
experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
experiment_df.T

由于实验是在培训脚本中创建的，要删除实验，您可以使用`list()`方法获取项目中所有实验，然后按实验名称进行过滤。

In [ ]:
experiments = aiplatform.Experiment.list()
for experiment in experiments:
    if experiment.name == EXPERIMENT_NAME:
        experiment.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的单个资源。

In [ ]:
# There are no resources to cleanup